# Algoritmo Value Iteration
En el presente código se implementa el algoritmo **Value Iteration** para la resolución del entorno 'Frozen Lake v1'. Se implementan dos versiones, la primera con un agente que tiene acceso a la función de transición y a las recompensas del entorno, y un segundo que no tiene este acceso, por lo que debe realizar estimaciones.

**NOTA IMPORTANTE:** Se logró reconocer que el método implementado en el libro falla en ocasiones; pues se "atora" en alguna acción específica, impidiéndole explorar nuevas acciones y estados, lo que provoca que el entrenamiento del agente entre a un bucle infinito (o demasiado largo). Para evitar esto, se inicializaron los valores para el primer *agente con conocimiento* en valores aleatorios, pero aún así no se logra evitar del todo. En fin, si se corre este código y alguna celda de entrenamiento tarda demasiado (más de un minuto), probablemente se haya estancado.

## Consideraciones del entorno y ecuaciones que se usarán

El algoritmo **Value Iteration** permite encontrar los valores óptimos V(s) para cada estado, de tal manera que al calcular el valor de acción máximo Q(s) de cada estado se pueda elegir la mejor acción en ese estado.

Consideraciones:

El entorno para el que se implementa el método en el presente documento, es para el entorno 'Frozen Lake v1' en su modo *slippery*, por lo que se deduce que es un entorno estocástico con políticas deterministas. Más concretamente:

- Política: 0.25 de probabilidad de elegir cualquiera de las 4 acciones (0, 1, 2, 3)
- Entorno: Posibilidad de caer en dos estados diferentes más a parte del principal, con una probabilidad de 0.33333 para cada uno

De aquí se deduce que las ecuaciones a usar para encontrar el V*(s) óptimo de cada estado son las siguientes, dadas su política determinista y entorno estocástico:

La ecuación de valor de acción (para entornos estocásticos) es:
$$ Q(s) = \sum_{s'}^{}P(s'|s,a) [R(s,a,s')+\gamma V(s')] $$

La ecuación de valor de estado será por tanto: 
$$ V_{*}(s) = max_{a}Q_{*}(s,a)$$

Se puede observar en esta última, que se requiere el valor de $Q_{*}$, si se revisa el libro se puede observar que la ecuación de valor de acción que se usa $ Q_{s}$ es equivalente a la función de valor de acción óptima $ Q_{*} $

## Agente con conocimiento del entorno

#### Clases y funciones a usar

Librerías

In [102]:
import numpy as np
import gymnasium as gym

El agente en este notebook ya incorpora el método **Value Iteration** para aprender a resolver el entorno

In [103]:
class Agente:
    def __init__(self, GAMMA, size_ambiente="4x4"):
        self.env = gym.make('FrozenLake-v1', is_slippery=True, map_name=size_ambiente)
        self.GAMMA = GAMMA
        self.V = np.random.rand(self.env.observation_space.n) # Esta variable contendrá el valor de cada estado
    
    def calcular_Q(self, s, a): # s=estado, a=acción
        # Calcular Q(s)
        valor_accion = sum([probabilidad*(recompensa + self.GAMMA*self.V[siguiente_estado]) for probabilidad, siguiente_estado, recompensa, _ in self.env.P[s][a]])
        return valor_accion
    
    def seleccionar_accion(self, s):
        mejor_accion = mejor_valor = False
        
        for accion in range(self.env.action_space.n): # Para cada acción (son 4)
            valor_accion = self.calcular_Q(s, accion)
            if mejor_valor is False or mejor_valor < valor_accion: # Si no hay mejor valor o el valor de la acción es mayor que el mejor valor
                mejor_valor = valor_accion
                mejor_accion = accion

        return mejor_accion
    
    def Value_Iteration(self):
        for estado in range(self.env.observation_space.n):
            # Calcular V*(s)
            valores_estado = [self.calcular_Q(estado, accion) for accion in range(self.env.action_space.n)] 
            self.V[estado] = max(valores_estado)
            
        return self.V

Esta función corre un número de episodios y regresa el porcentaje de victorias

In [104]:
def revisar_mejoras(agente, no_episodios = 20, size_ambiente="4x4"):
    ambiente_test = gym.make('FrozenLake-v1', is_slippery=True, render_mode='rgb_array', map_name=size_ambiente)
    recompensa_total_episodios = 0
    
    for n in range(no_episodios):
        recompensa_episodio = 0
        estado = ambiente_test.reset()[0]
        while True:
            accion = agente.seleccionar_accion(estado)
            new_estado, new_recompensa, is_done, truncado, info = ambiente_test.step(accion)
            recompensa_episodio += new_recompensa
            if is_done is True:
                break
            estado = new_estado
        recompensa_total_episodios += recompensa_episodio
    recompensa_total_episodios /= no_episodios
    return recompensa_total_episodios

# Revisar mejoras con render humano
def revisar_mejoras_render(agente, no_episodios = 20, size_ambiente="4x4"):
    ambiente_test = gym.make('FrozenLake-v1', is_slippery=True, render_mode='human', map_name=size_ambiente)
    recompensa_total_episodios = 0
    
    for n in range(no_episodios):
        recompensa_episodio = 0
        estado = ambiente_test.reset()[0]
        while True:
            accion = agente.seleccionar_accion(estado)
            new_estado, new_recompensa, is_done, truncado, info = ambiente_test.step(accion)
            recompensa_episodio += new_recompensa
            if is_done is True:
                break
            estado = new_estado
        recompensa_total_episodios += recompensa_episodio
    recompensa_total_episodios /= no_episodios
    return recompensa_total_episodios

Funciones para extraer la política aprendida del agente y para poder visualizarla mejor

In [105]:
# Extraer política
def extraer_politica(agente):
    politica = np.zeros(agente.env.observation_space.n)
    for estado in range(agente.env.observation_space.n):
        politica[estado] = agente.seleccionar_accion(estado) # En este caso se selecciona la acción que se realiza siempre en ese estado
                                                                # con los actuales valores de estado V
    return politica

# Visualizar política
def visualizar_politica(politica):
    print(politica.reshape(-1,4))
    print('\n')
    ayuda_visual = {0:'<', 1:'v', 2:'>', 3:'^'}
    flechas_politica = [ayuda_visual[x] for x in politica]
    print(np.array(flechas_politica).reshape(-1,4))

#### Entrenamiento

In [106]:
UMBRAL_RECOMPENSA = 0.9
t = 0
mejor_recompensa = 0
agente = Agente(GAMMA=0.95)

# BUCLE DE ENTRENAMIENTO
while mejor_recompensa < UMBRAL_RECOMPENSA:
    agente.Value_Iteration()
    t += 1
    recompensa_total_episodios = revisar_mejoras(agente)
    
    if recompensa_total_episodios > mejor_recompensa:
        print(f"Mejor recompensa actualizada {recompensa_total_episodios:.2f} en la iteración {t}")
        mejor_recompensa = recompensa_total_episodios

Mejor recompensa actualizada 0.90 en la iteración 1


Valores de estado finales calculados por el método **Value Iteration**

In [107]:
print(agente.V)

[0.62712855 0.51930681 0.63051944 0.6422135  0.76176898 0.02751469
 0.77339044 0.84516176 0.657069   0.78639335 0.66925934 0.33776129
 0.353127   0.71066851 1.03836723 0.80417024]


#### Análisis del comportamiento adquirido

Visualizar la política adquirida, observar que es una política determinista. Esto gracias a que se calculó el valor de estado óptimo para cada estado, de tal manera que al seleccionar la acción con el valor máximo en ese estado concreto, siempre fuera la misma.

In [108]:
politica = extraer_politica(agente)
visualizar_politica(politica)

[[0. 3. 2. 2.]
 [0. 0. 2. 0.]
 [3. 1. 0. 0.]
 [0. 2. 1. 0.]]


[['<' '^' '>' '>']
 ['<' '<' '>' '<']
 ['^' 'v' '<' '<']
 ['<' '>' 'v' '<']]


Visualizar algunos episodios

In [109]:
# revisar_mejoras_render(agente)

#### Análisis con Tensorboard
Esta parte del código sólo funciona en Google Colab

In [110]:
# from torch.utils.tensorboard import SummaryWriter
# writer = SummaryWriter()

In [111]:
# %load_ext tensorboard
# %tensorboard --logdir runs

In [112]:
# agente2 = Agente(GAMMA=0.95)
# t = 0
# mejor_recompensa = 0

# while mejor_recompensa < UMBRAL_RECOMPENSA:
#     agente2.Value_Iteration()
#     t += 1
#     recompensa_total_episodios = revisar_mejoras(agente2)
#     writer.add_scalar('Recompensa', recompensa_total_episodios, t)
#     if recompensa_total_episodios > mejor_recompensa:
#         mejor_recompensa = recompensa_total_episodios
        
# writer.close()

#### Comportamiento del agente en un entorno más grande (Frozen Lake v1 8x8)
**NOTA IMPORTANTE:** Cabe destacar que para que el entrenamiento no se quedara en un bucle infinito para este entorno, se tuvo que hacer un cambio a la clase, e inicializar los valores de estado **V** aleatoriamente y no en ceros como decía en el libro

In [113]:
agenteGrande = Agente(GAMMA=0.95, size_ambiente="8x8")
UMBRAL_RECOMPENSA = 0.9
t = 0
mejor_recompensa = 0

# BUCLE DE ENTRENAMIENTO
while mejor_recompensa < UMBRAL_RECOMPENSA:
    agenteGrande.Value_Iteration()
    t += 1
    recompensa_total_episodios = revisar_mejoras(agenteGrande, size_ambiente="8x8")
    
    if recompensa_total_episodios > mejor_recompensa:
        print(f"Mejor recompensa actualizada {recompensa_total_episodios:.2f} en la iteración {t}")
        mejor_recompensa = recompensa_total_episodios

Mejor recompensa actualizada 0.20 en la iteración 25
Mejor recompensa actualizada 0.40 en la iteración 28
Mejor recompensa actualizada 0.45 en la iteración 29
Mejor recompensa actualizada 0.50 en la iteración 30
Mejor recompensa actualizada 0.65 en la iteración 36
Mejor recompensa actualizada 0.70 en la iteración 43
Mejor recompensa actualizada 0.75 en la iteración 44
Mejor recompensa actualizada 0.85 en la iteración 51
Mejor recompensa actualizada 0.95 en la iteración 71


Correr algunos episodios con render

In [114]:
# revisar_mejoras_render(agenteGrande, size_ambiente="8x8", no_episodios=50)

## Agente con estimaciones

#### Clases y funciones

In [115]:
import collections

class AgenteEstimaciones:
    def __init__(self, GAMMA, size_ambiente="4x4"):
        self.env = gym.make('FrozenLake-v1', is_slippery=True, render_mode='rgb_array', map_name=size_ambiente)
        self.GAMMA = GAMMA
        self.state = self.env.reset()[0]
        self.recompensas = collections.defaultdict(float)
        self.transiciones = collections.defaultdict(collections.Counter)
        self.V = np.zeros(self.env.observation_space.n)
        
    def reconocimiento(self, contador): # Este método se encarga de reconocer el entorno y de actualizar las estimaciones de recompensa y transiciones
        for n in range(contador):
            accion = self.env.action_space.sample()
            nuevo_estado, recompensa, is_done, truncado, info = self.env.step(accion) # Se realiza una acción aleatoria
            self.recompensas[(self.state, accion, nuevo_estado)] = recompensa
            self.transiciones[(self.state, accion)][nuevo_estado] += 1
            if is_done:
                self.state = self.env.reset()[0]
            else:
                self.state = nuevo_estado
                
    def calcular_Q(self, estado, accion):
        contador_transiciones = self.transiciones[(estado, accion)]
        total = sum(contador_transiciones.values()) # Suma de todas las transiciones desde el estado y acción actuales
        valor_accion = 0.0
        
        for siguiente_estado, contador in contador_transiciones.items():
            r = self.recompensas[(estado, accion, siguiente_estado)] # Se obtiene la recompensa estimada
            prob = (contador / total)                               # Se obtiene la probabilidad estimada
            valor_accion += prob * (r + self.GAMMA * self.V[siguiente_estado]) # Calcular Q
        
        return valor_accion
    
    def seleccionar_accion(self, s):
        mejor_accion = mejor_valor = False
        
        for accion in range(self.env.action_space.n): # Para cada acción (son 4)
            valor_accion = self.calcular_Q(s, accion)
            if mejor_valor is False or mejor_valor < valor_accion: # Si no hay mejor valor o el valor de la acción es mayor que el mejor valor
                mejor_valor = valor_accion
                mejor_accion = accion

        return mejor_accion
    
    def Value_Iteration(self, contador=100):
        self.reconocimiento(contador)
        for estado in range(self.env.observation_space.n):
            valores_estado = [self.calcular_Q(estado, accion) for accion in range(self.env.action_space.n)]
            self.V[estado] = max(valores_estado)

#### Entrenamiento

In [116]:
UMBRAL_RECOMPENSA = 0.9
t = 0
mejor_recompensa = 0
agenteEsti = AgenteEstimaciones(GAMMA=0.95)

# BUCLE DE ENTRENAMIENTO
while mejor_recompensa < UMBRAL_RECOMPENSA:
    agenteEsti.Value_Iteration()
    t += 1
    recompensa_total_episodios = revisar_mejoras(agenteEsti)
    
    if recompensa_total_episodios > mejor_recompensa:
        print(f"Mejor recompensa actualizada {recompensa_total_episodios:.2f} en la iteración {t}")
        mejor_recompensa = recompensa_total_episodios

Mejor recompensa actualizada 0.25 en la iteración 11
Mejor recompensa actualizada 0.40 en la iteración 21
Mejor recompensa actualizada 0.50 en la iteración 23
Mejor recompensa actualizada 0.75 en la iteración 25
Mejor recompensa actualizada 0.80 en la iteración 27
Mejor recompensa actualizada 0.90 en la iteración 40


Visualizar algunos episodios del agente ya entrenado

In [117]:
# revisar_mejoras_render(agenteEsti, no_episodios=50)

Visualizar las estimaciones de recompensa

In [118]:
agenteEsti.recompensas

defaultdict(float,
            {(0, 1, 1): 0.0,
             (1, 1, 5): 0.0,
             (0, 0, 4): 0.0,
             (4, 3, 5): 0.0,
             (0, 0, 0): 0.0,
             (1, 2, 1): 0.0,
             (1, 0, 0): 0.0,
             (0, 3, 0): 0.0,
             (1, 2, 2): 0.0,
             (2, 0, 2): 0.0,
             (2, 2, 6): 0.0,
             (6, 2, 7): 0.0,
             (0, 1, 0): 0.0,
             (0, 2, 0): 0.0,
             (0, 3, 1): 0.0,
             (1, 0, 5): 0.0,
             (0, 1, 4): 0.0,
             (4, 0, 4): 0.0,
             (4, 1, 4): 0.0,
             (4, 1, 5): 0.0,
             (0, 2, 1): 0.0,
             (1, 1, 2): 0.0,
             (2, 1, 1): 0.0,
             (1, 3, 1): 0.0,
             (1, 3, 0): 0.0,
             (1, 2, 5): 0.0,
             (2, 1, 6): 0.0,
             (0, 2, 4): 0.0,
             (4, 2, 0): 0.0,
             (1, 0, 1): 0.0,
             (4, 3, 0): 0.0,
             (4, 2, 5): 0.0,
             (6, 1, 10): 0.0,
             (10, 3, 11

Visualizar las estimaciones de transición

In [119]:
agenteEsti.transiciones

defaultdict(collections.Counter,
            {(0, 1): Counter({1: 126, 0: 132, 4: 135}),
             (1, 1): Counter({5: 50, 2: 70, 0: 60}),
             (0, 0): Counter({4: 113, 0: 264}),
             (4, 3): Counter({5: 57, 0: 54, 4: 35}),
             (1, 2): Counter({1: 60, 2: 50, 5: 66}),
             (1, 0): Counter({0: 48, 5: 44, 1: 41}),
             (0, 3): Counter({0: 267, 1: 140}),
             (2, 0): Counter({2: 30, 6: 28, 1: 30}),
             (2, 2): Counter({6: 21, 2: 33, 3: 27}),
             (6, 2): Counter({7: 8, 10: 4, 2: 11}),
             (0, 2): Counter({0: 151, 1: 142, 4: 162}),
             (4, 0): Counter({4: 43, 0: 54, 8: 51}),
             (4, 1): Counter({4: 53, 5: 45, 8: 59}),
             (2, 1): Counter({1: 31, 6: 21, 3: 31}),
             (1, 3): Counter({1: 65, 0: 51, 2: 50}),
             (4, 2): Counter({0: 48, 5: 45, 8: 53}),
             (2, 3): Counter({3: 30, 1: 20, 2: 25}),
             (3, 0): Counter({3: 18, 2: 18, 7: 11}),
             (3, 1